# Collatz conjecture - Problem 14 - Project Euler

Explanation of the conjecture
Which starting point n (less than 1 million) produces the longest Collatz chain?

\begin{equation}
if \quad n \% 2 = 0
\quad n = \frac{n}{2}
\end{equation}

\begin{equation}
if \quad n \% 2 \neq 0
\quad n = 3n + 1
\end{equation}

## CPU Python implementation

In [1]:
import numpy as np

class Collatz(object):
    " Computes the length of the Collatz chain for all starting points less than N "
    def __init__(self, N):
        self.N = N
        self.starts = np.arange(1, N)
        self.lengths = []
        self.max_chain = 1

    def collatz_sequence(self, n):
        if n%2 == 0:
            return n/2
        else:
            return 3*n + 1

    def length_sequence(self, n):
        " Calculates the length of the Collatz chain for a given n"
        counter = 0
        while n != 1:
            n = self.collatz_sequence(n)
            counter += 1
        return counter

    def longest_chain(self):
        i_arg = np.argsort(self.lengths)
        self.max_chain = self.lengths[i_arg[-1]]
        
        self.max_start = self.starts[i_arg[-1]]
        

    def __call__(self):
        for n in self.starts:
            # Computes the lengths sequentially for all starting points n = [1, ..., N]
            length = self.length_sequence(n)
            self.lengths.append(length)
        self.lengths = np.array(self.lengths)

        self.longest_chain()

In [2]:
from time import time as timer
Nmax = 100000
collatz = Collatz(N=Nmax)
start_cpu = timer()
collatz()
end_cpu = timer()
speed_cpu = end_cpu - start_cpu
print('Longest Collatz chain for n < %d :' %collatz.N, collatz.max_chain)
print('For a starting point: %d' %collatz.max_start)
print('CPU Python code took %f seconds' %speed_cpu)

Longest Collatz chain for n < 100000 : 350
For a starting point: 77031
CPU Python code took 8.633451 seconds


# GPU PyCUDA implementation

In [3]:
import pycuda.autoinit
import pycuda.driver as drv
import pycuda.gpuarray as gpuarray
from pycuda.compiler import SourceModule

mod = SourceModule("""
__global__ void collatz (int *n){
const int i = blockIdx.x * blockDim.x * blockDim.y + threadIdx.y * blockDim.x + threadIdx.x;

int m = n[i];
int counter = 0;

while (m > 1){
    if (m % 2 == 0){
        m = m / 2;
    }
    else {
        m = 3*m + 1; 
    }
    counter += 1;
}
n[i] = counter;
}
""")

collatz_gpu = mod.get_function("collatz")

ModuleNotFoundError: No module named 'pycuda'

In [4]:
n = np.arange(1, Nmax).astype(np.int32)
nn = n.copy()
start_gpu = timer()

# Allocate GPU memory
n_gpu = drv.mem_alloc(n.nbytes)

# Transfer to GPU
drv.memcpy_htod(n_gpu, nn)

# Select the amount of blocks - DEVICE DEPENDENT!!
N = n.shape[0]
if N <= 1024:
    Nblocks = N
    Ngrid = 1
else:
    Nblocks = 1024
    Ngrid = int(N/Nblocks)
    if (N % Nblocks) != 0:
        Ngrid += 1

# Kernel call
collatz_gpu(n_gpu, block=(Nblocks,1,1), grid=(Ngrid,1))

# Transfer back to CPU
drv.memcpy_dtoh(nn, n_gpu)

# Free GPU memory
n_gpu.free()
end_gpu = timer()
speed_gpu = end_gpu - start_gpu

# Check results with the CPU version
err = collatz.lengths - nn
print('CPU - GPU discrepancy: %f' %(np.mean(err**2)))
print('GPU PyCUDA code took %f seconds' %speed_gpu)
ratio = speed_cpu / speed_gpu
print('%d times faster than CPU Python' %ratio)

NameError: name 'drv' is not defined